In [1]:
from app import Input, Inputs

In [2]:
input = Input(brewery_name='good',review_profilename='me',review_year=2020,review_month=1,review_appearance=1,review_aroma=1,review_overall=1,review_palate=1,review_taste=1,beer_abv=1)
inputs = Inputs(input_list=[input, input])

In [3]:
from fastapi import FastAPI
from starlette.responses import JSONResponse
from joblib import load
from app import get_device, load_pytorch_model, MyDataset

In [4]:
app = FastAPI()

# get device
device = get_device()
# load model
model = load_pytorch_model('./models/pytorch_model.pth', device)
# load map for labels
label_map = load('./models/label_map')
# load preprocesser
preprocessor = load('./models/preprocessor')
# define features
features = Input.schema()['properties']

cpu
cpu


In [5]:
def prepare_features(df):
    from torch.utils.data import DataLoader
    # preprocess data
    preprocessed = preprocessor.transform(df)
    # create dataset
    dataset = MyDataset(preprocessed)
    # create data loader
    data_loader = DataLoader(dataset, batch_size=32, shuffle=False)
    return data_loader

def predict(data_loader):
    import torch
    with torch.no_grad():
        for features in data_loader:
            # input features
            features = features.float()
            features = features.to(device)
        # predict
        outputs = model(features)
        # number of dims, to account for single sample
        dims = len(outputs.shape)
        # get predicted classes
        _, predicted = torch.max(outputs.data, dims-1)
    return predicted.numpy()

In [6]:
import pandas as pd
# input to dataframe
df = pd.DataFrame([input.dict()])
# prepare features
data_loader = prepare_features(df)

In [11]:
from joblib import dump
for feature in data_loader:
    dump(feature, './models/example_input_tensor')

In [119]:
# inputs to dataframe
df = pd.DataFrame(inputs.dict_inputs())
# prepare features
data_loaders = prepare_features(df)

In [130]:
prediction = predict(data_loader)
prediction

array(77)

In [131]:
import numpy as np
np.vectorize(label_map.get)(prediction).tolist()

'Low Alcohol Beer'

In [28]:
import torch
torch.max(prediction.data, 1)

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [95]:
torch.max(outputs.data,0)

torch.return_types.max(
values=tensor([  4.8411,   8.8629,  -2.6552,  14.6094, -75.8212,   6.9101,  17.9016,
          7.9461,   6.2154, -31.5230, -23.4660, -37.0261,   8.1845,  -7.5332,
          7.5206,  16.2280,   8.0333, -16.1123,   8.2967,  -9.3449,  11.9510,
         -8.0741, -13.9905,   2.2087,  13.6348, -69.8907, -51.1821,  24.5593,
          9.9829, -11.9452,  -4.0773,   9.8940,   1.0742,   8.3843,   7.1419,
          9.7032,  -0.9713, -17.6717,  -3.5996,  15.3207,  -7.6790, -24.1883,
        -29.9908,  15.7601,  11.4170,  19.9726,  13.8423,   1.7244,  17.5218,
         -1.5563,   4.5678, -10.5625,  10.8110,  10.1420, -12.2132,   3.9774,
          6.1423,  -1.4292, -11.6013,  -4.6609,  22.2868,  -3.1780,   6.5762,
         -8.2519,  12.4662,   7.3707,  23.3990,  16.9549,   9.2236,  18.5803,
          8.7841,  10.9538,  25.6658,  20.1544,  18.2914,   6.4373,  25.0564,
         29.6465, -24.8069,   2.7250,   7.2780,  18.6077,   5.6667,   5.7901,
         -9.8382,   4.1664, -19.7